<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# Eikon Data API

**Symbology**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers

* `RICs`(Reuters Instruments Codes)
* Symbology conversions
* Working with `RICs`
* Working with data fields
* Working with Chain `RICs`

## Imports and Versions

The following imports several **packages** as used in the following.

In [2]:
import eikon as ek  # the Eikon Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline

The following **Python and package versions** are used.

In [3]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [3]:
ek.__version__

'0.1.10'

In [4]:
np.__version__

'1.13.3'

In [5]:
pd.__version__

'0.22.0'

In [6]:
cf.__version__

'0.12.1'

## Connecting to Eikon Data API

This code sets the `app_id` to connect to the **Eikon Data API Proxy** which needs to be running locally. It requires the previously created text file `eikon.cfg` to be in the current working directory.

In [4]:
import eikon as ek
ek.set_app_key('0ed6a35e0937415eab446d3375bca7cf671d6b4c')

In [1]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

NameError: name 'cp' is not defined

In [8]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

## RICs

A `RIC` is a **Reuters Instrument Code**.  It is used, amongst other things, to specify which instruments to retrieve historical data or, for example, news headlines. The following represents a list of `RICs` across several asset classes.

In [17]:
rics = [
    'GE',  # General Electric stock
    'AAPL.O',  # Apple stock
    '.SPX',  # S&P 500 stock index
    '.VIX',  # VIX volatility index
    'EUR=',  # EUR/USD exchange rate
    'XAU=',  # Gold price
    'DE10YT=RR',  # 10yr Bund price
]

`RICs` can be searched for in the **Data Item Browser**.

<img src="http://eikon.tpq.io/tr_eikon_dib_rics.png">

## Symbology Conversions

As there are a number of different symbology types (eg ISIN, SEDOL etc) used in financial markets &mdash; the ability to map between these different schemes is very useful. The function `ek.get_symbology()` allows the conversion from one type of symbol to another one.

In [16]:
help(ek.get_symbology)

Help on function get_symbology in module eikon.symbology:

get_symbology(symbol, from_symbol_type='RIC', to_symbol_type=None, raw_output=False, debug=False, bestMatch=True)
    Returns a list of instrument names converted into another instrument code.
    For example: convert SEDOL instrument names to RIC names
    
    Parameters
    ----------
    symbol: string or list of strings
        Single instrument or list of instruments to convert.
    
    from_symbol_type: string
        Instrument code to convert from.
        Possible values: 'CUSIP', 'ISIN', 'SEDOL', 'RIC', 'ticker', 'lipperID', 'IMO'
        Default: 'RIC'
    
    to_symbol_type: string or list
        Instrument code to convert to.
        Possible values: 'CUSIP', 'ISIN', 'SEDOL', 'RIC', 'ticker', 'lipperID', 'IMO', 'OAPermID'
        Default: None  (means all symbol types are requested)
    
    raw_output: boolean, optional
        Set this parameter to True to get the data in json format
        if set to False, 

In [23]:
rics[:2]

['GE', 'AAPL.O']

For example, the `RICs` can be converted to `ISINs` (International Securities Identification Numbers).

In [20]:
ek.get_symbology(rics[:2], from_symbol_type='RIC', to_symbol_type='ISIN')

,ISIN
GE,US3696041033
AAPL.O,US0378331005


Multilpe output types can also be specified.

In [24]:
ek.get_symbology(rics[:2], from_symbol_type='RIC', to_symbol_type=['ISIN', 'ticker'])

,ISIN,ticker
GE,US3696041033,GE
AAPL.O,US0378331005,AAPL


There is no guarantee, however, that a conversion is possible.

In [25]:
ek.get_symbology(rics, from_symbol_type='RIC', to_symbol_type=['ISIN', 'ticker'])

,ISIN,ticker,error
GE,US3696041033,GE,NaN
AAPL.O,US0378331005,AAPL,NaN
.SPX,US78378X1072,SPX,NaN
.VIX,NaN,VIX,NaN
EUR=,NaN,NaN,No best match available
XAU=,NaN,NaN,No best match available
DE10YT=RR,DE0001102499,NaN,NaN


Of course, other symbol types, such as `SEDOL` ([Stock Exchange Daily Official List](http://www.londonstockexchange.com/products-and-services/reference-data/sedol-master-file/documentation/sedol-technical-specification.pdf)) can also be converted **to** `RICs` &mdash; or `ISINs` to this end.

In [26]:
sedols = ['B1YW440', '0673123', 'B02J639', 'B1XZS82', '0045614', '0053673',
          '0989529', '0216238', '0969703', '0263494']

In [27]:
ek.get_symbology(sedols, from_symbol_type='SEDOL', to_symbol_type=['RIC', 'ISIN'])

,ISIN,RIC
B1YW440,GB00B1YW4409,III.L
0673123,GB0006731235,ABF.L
B02J639,GB00B02J6398,ADML.L
B1XZS82,GB00B1XZS820,AAL.L
0045614,GB0000456144,ANTO.L
0053673,GB0000536739,AHT.L
0989529,GB0009895292,AZN.L
0216238,GB0002162385,AV.L
0969703,GB0009697037,BAB.L
0263494,GB0002634946,BAES.L


## Working with RICs

Historical data shall be retrieved for the following stocks.

In [28]:
symbols = ['US0378331005', 'US0231351067', 'US30303M1027', 'US4581401001']

The `ISINs` are easily converted to `RICs`.

In [29]:
rics = ek.get_symbology(symbols, from_symbol_type='ISIN', to_symbol_type='RIC')

In [32]:
rics

,RIC
US0378331005,AAPL.O
US0231351067,AMZN.O
US30303M1027,FB.O
US4581401001,INTC.O


With the `RICs` available, one can then retrieve historical data.

In [34]:
rics = list(rics.RIC.values)
rics

['AAPL.O', 'AMZN.O', 'FB.O', 'INTC.O']

In [35]:
data = ek.get_timeseries(rics,  # the RICs
                         fields='CLOSE',  # close field
                         start_date='2017-10-01',  # start date
                         end_date='2018-01-31')  # end date

In [36]:
data.tail()  # final five rows

CLOSE,AAPL.O,AMZN.O,FB.O,INTC.O
Date,,,,
2018-01-25,171.11,1377.95,187.48,45.30
2018-01-26,171.51,1402.05,190.00,50.08
2018-01-29,167.96,1417.68,185.98,49.98
2018-01-30,166.97,1437.82,187.12,48.79
2018-01-31,167.43,1450.89,186.89,48.14


In [37]:
data.info()  # DataFrame meta information

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 84 entries, 2017-10-02 to 2018-01-31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL.O  84 non-null     float64
 1   AMZN.O  84 non-null     float64
 2   FB.O    84 non-null     float64
 3   INTC.O  84 non-null     float64
dtypes: float64(4)
memory usage: 3.3 KB


The code that follows visualizes the **time series data** in the `CLOSE` column as a **line plot**.

In [38]:
data.normalize().iplot()

One can also retrieve **news headlines** for these `RICs`.

In [24]:
','.join(rics)

'AMZN.O,AAPL.O,FB.O,INTC.O'

In [25]:
ek.get_news_headlines('R:{}'.format(','.join(rics)),  # the RICs
                      date_from='2018-02-02T20:00:00',  # the starting time
                      date_to='2018-02-02T22:00:00',  # the end time
                      count=20)  # number of headlines

,versionCreated,text,storyId,sourceCode
2018-02-02 21:54:31.364,2018-02-02 21:54:31.364,"The Amazing Protein Company, Inc. Continues to...",urn:newsml:reuters.com:20180202:nNRA5h7equ:1,NS:ACSWIR
2018-02-02 21:48:00.462,2018-02-02 21:48:00.462,INTEL CORP SEC Filings files Form -- 4,urn:newsml:reuters.com:20180202:nEOL4Vlbfx:1,NS:EDG
2018-02-02 21:46:06.943,2018-02-02 21:46:06.943,Reuters Insider - Dow sinks 665 points,urn:newsml:reuters.com:20180202:nRTV2HS6Qh:1,NS:RTRS
2018-02-02 21:41:56.953,2018-02-02 21:41:56.953,Reuters Insider - Dow posts biggest percentage...,urn:newsml:reuters.com:20180202:nRTVbPbv0t:1,NS:CNBC
2018-02-02 21:41:17.405,2018-02-02 21:41:17.405,INTEL CORP SEC Filings files Form -- 4,urn:newsml:reuters.com:20180202:nEOL1BYqd3:1,NS:EDG
2018-02-02 12:09:47.000,2018-02-02 21:38:48.000,UPDATE 3-Bitcoin skids amid broad cryptocurren...,urn:newsml:reuters.com:20180202:nL8N1PS2HM:1,NS:RTRS
2018-02-02 21:33:19.576,2018-02-02 21:33:19.576,"Not just iPhones, Mac and iPads too drive Appl...",urn:newsml:reuters.com:20180202:nNRA5h7f99:1,NS:ECOTIM
2018-02-02 21:32:45.000,2018-02-02 21:32:45.000,Tillerson advierte a México vigilar intervenci...,urn:newsml:reuters.com:20180202:nL2N1PS1ZY:1,NS:RTRS
2018-02-02 21:30:46.225,2018-02-02 21:30:46.225,Reuters Insider - Nasdaq down lower on big-cap...,urn:newsml:reuters.com:20180202:nRTVz8KR6:1,NS:CNBC
2018-02-02 21:28:53.000,2018-02-02 21:28:53.000,Exposición a radiación por radiofrecuencia est...,urn:newsml:reuters.com:20180202:nL2N1PS1UR:1,NS:RTRS


## Working with Data Fields

The Eikon Data API provides access to many different **data fields** for single or multiple instruments. Field names can be searched for in the **Data Item Browser**. There are several ways of doing this &mdash; the filter button opens a high level menu if you want to see all fields in a particular category. If you know what you are looking for, say `Revenue` you can just type this in and fields will be suggested. If you have typed in an instrument before you will also see the values associated with the fields. This is very useful in finding the exact field you want.

<img src="http://eikon.tpq.io/tr_eikon_dib_fields.png">

Consider the Apple, Inc. stock with `RIC` `AAPL.O`. For the this instrument, different fields of data shall be retrieved. The function to be used is `ek.get_data()`.

In [26]:
help(ek.get_data)

Help on function get_data in module eikon.data_grid:

get_data(instruments, fields, parameters=None, field_name=False, raw_output=False, debug=False)
    Returns a pandas.DataFrame with fields in columns and instruments as row index
    
    Parameters
    ----------
    instruments: string or list
        Single instrument or list of instruments to request.
    
    fields: string, dictionary or list of string and/or dictionary
        - Single field as a string: 'TR.PriceClose'
        - Single field as a dictionary for providing parameters and/or sort option:  
         ex:
            {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' }}
            {'TR.GrossProfit': { 'params':{ 'Scale': 6, 'Curn': 'EUR' },sort_dir:'desc'}
       - List of fields. Items could be strings and/or dictionaries     
           ex: 
             ['TR.PriceClose','TR.PriceOpen']
             [{'TR.PriceClose':  {'sort_dir':asc,sort_priority:1}},{'TR.PriceOpen':  {'sort_dir':asc,sort_priority:0}}


In [39]:
data, err = ek.get_data('AAPL.O', ['TR.PriceClose', 'TR.Volume', 'TR.PriceLow',
                                        'TR.TotalReturnYTD', 'TR.TotalReturn52WK'])

In [40]:
data

,Instrument,Price Close,Volume,Price Low,YTD Total Return,52 Week Total Return
0,AAPL.O,282.97,31627183,277,-3.405131,40.331787


In [41]:
err  # no error

The same methodology works for multiple `RICs`.

In [42]:
rics

['AAPL.O', 'AMZN.O', 'FB.O', 'INTC.O']

In [43]:
data, err = ek.get_data(rics, ['TR.PriceClose', 'TR.Volume', 'TR.PriceLow',
                                        'TR.TotalReturnYTD', 'TR.TotalReturn52WK'])

In [44]:
data

,Instrument,Price Close,Volume,Price Low,YTD Total Return,52 Week Total Return
0,AAPL.O,282.97,31627183,277.000,-3.405131,40.331787
1,AMZN.O,2410.22,3831797,2382.000,30.434453,23.561106
2,FB.O,190.07,28868528,180.825,-7.395859,-0.741553
3,INTC.O,59.26,50220505,56.100,-0.498770,15.677123


In [45]:
cols = ['YTD Total Return', '52 Week Total Return']
data.set_index('Instrument')[cols].iplot(kind='bar')

## Working with Chain RICs

The `RIC` structure also provide access to groups of `RICs` that have a common association &mdash; for example, the constituent members of the FTSE 100 index or a list of options for a particular underlying contract. These groups are maintained by Thomson Reuters and are always up to date. It is important to note that chains contain a list of instruments only and not their values. 

For example:

* `0#.FTSE` gives you a list of FTSE 100 constituents 

* `0#GDAX*.EX` gives you a list of all options for all maturities for the DAX index
   
* `0#GDAXM8*.EX` gives you a list of all options for June 2018 maturity for the DAX index

Via Chain `RICs` it is convenient to retrieve data for e.g. for **all constituents of an index** at once.

In [61]:
dax = ek.get_data('0#.GDAXI', fields=['TR.CommonName', 'TR.PriceClose',
                                      'TR.Volume', 'TR.TotalReturnYTD'])

In [65]:
dax[1]

In [66]:
dax[0].set_index('Instrument')['YTD Total Return'].sort_values().iplot(kind='bar', color='blue')

## Conclusions

This tutorial covers:

* `RICs`(Reuters Instruments Codes)
* Symbology conversions
* Working with `RICs`
* Working with data fields
* Working with Chain `RICs`

The Eikon Data API allows for **easy symbology conversions** and gives access to a **wealth of data fields** per instrument. The Data Item Browser allows to search for both `RICs` as well as data fields &mdash; and also provides values as well so you can see if it is the right field for you or not.

## Eikon Data API Developer Resources

* [Overview](https://developers.thomsonreuters.com/eikon-data-apis) 
* [Quick Start ](https://developers.thomsonreuters.com/eikon-data-apis/quick-start)
* [Documentation](https://developers.thomsonreuters.com/eikon-data-apis/docs)
* [Downloads](https://developers.thomsonreuters.com/eikon-data-apis/downloads)
* [Tutorials](https://developers.thomsonreuters.com/eikon-data-apis/learning)
* [Q&A Forums](https://developers.thomsonreuters.com/eikon-data-apis/qa) 

Data Item Browser Application: Type `DIB` into Eikon Search Bar.

* [Article on Chains](https://developers.thomsonreuters.com/article/simple-chain-objects-ema-part-1)

<img src="http://eikon.tpq.io/refinitiv_logo.png" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">